In [1]:
import os

In [2]:
%pwd

'd:\\sign_language_conversion\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\sign_language_conversion'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Rahulagowda004/sign_language_conversion.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Rahulagowda004"
os.environ["MLFLOW_TRACKING_PASSWORD"]="d15121a9b37945a700eae5385a0ae54fa810a813"

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str

In [7]:
from sign_language_conversion.constants import *
from sign_language_conversion.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model=Path("model/model.pkl"),
            training_data=Path("artifacts/prepare_dataset/root_dir/data.pickle"),
            mlflow_uri="https://dagshub.com/Rahulagowda004/sign_language_conversion.mlflow",
            all_params=self.params
        )
        return eval_config

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import mlflow
import mlflow.sklearn
from urllib.parse import urlparse
import json
import pickle
import numpy as np

In [16]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    @staticmethod
    def load_model(path: Path):
        with open(path, 'rb') as file:
            model_dict = pickle.load(file)
        model = model_dict['model']
        return model

    def evaluation(self):
        with open(self.config.training_data, 'rb') as file:
            data_dict = pickle.load(file)
        
        data = np.asarray(data_dict['data'])
        labels = np.asarray(data_dict['labels'])

        x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, shuffle=True, stratify=labels)

        self.model = self.load_model(self.config.path_of_model)
        self.y_pred = self.model.predict(x_test)
        self.score = accuracy_score(y_test, self.y_pred)
        self.save_score()

    def save_score(self):
        scores = {"accuracy": self.score * 100}
        with open('scores.json', 'w') as file:
            json.dump(scores, file)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"accuracy": self.score})

            if tracking_url_type_store != "file":
                # Register the model
                mlflow.sklearn.log_model(self.model, "SGC_CONVERTER", registered_model_name="RANDOM_FOREST")
            else:
                mlflow.sklearn.log_model(self.model, "model")

In [17]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2024-05-25 19:23:13,025: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-25 19:23:13,028: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-25 19:23:13,031: INFO: common: created directory at: artifacts]


Successfully registered model 'RANDOM_FOREST'.
2024/05/25 19:23:43 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: RANDOM_FOREST, version 1
Created version '1' of model 'RANDOM_FOREST'.
